In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 定義數據路徑
data_dir = 'D:\Garbage classification'

# 定義圖像大小和批量大小
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 32

# 圖像生成器，用於數據增強和標準化
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,  # 旋轉範圍
    shear_range=0.15,  # 剪切範圍
    zoom_range=0.15,  # 縮放範圍
    horizontal_flip=True,  # 水平翻轉
    fill_mode='nearest',  # 填充模式
    validation_split=0.2  # 設置驗證集比例
)

# 創建訓練和驗證數據生成器
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=data_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical',
    subset='training'
)

val_data_gen = train_image_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical',
    subset='validation'
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# 構建CNN模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

# 編譯模型
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 顯示模型結構
model.summary()


Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │    58,982,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │         3,078 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,081,286 (225.38 MB)

 Trainable params: 59,080,262 (225.37 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [6]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 設置回調函數
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 訓練模型
history = model.fit(
    train_data_gen,
    epochs=20,
    steps_per_epoch=train_data_gen.samples // BATCH_SIZE,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // BATCH_SIZE,
    callbacks=[reduce_lr, early_stopping]
)


Epoch 1/20


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.3519 - loss: 1.8909 - val_accuracy: 0.2396 - val_loss: 1.8172 - learning_rate: 1.0000e-04
Epoch 2/20
 1/63 ━━━━━━━━━━━━━━━━━━━━ 1:54 2s/step - accuracy: 0.5312 - loss: 1.2271

D:\Anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5312 - loss: 1.2271 - val_accuracy: 0.1739 - val_loss: 1.9797 - learning_rate: 1.0000e-04
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.5042 - loss: 1.3731 - val_accuracy: 0.3562 - val_loss: 1.6210 - learning_rate: 1.0000e-04
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5625 - loss: 1.4021 - val_accuracy: 0.3043 - val_loss: 1.6605 - learning_rate: 1.0000e-04
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.5396 - loss: 1.3180 - val_accuracy: 0.3646 - val_loss: 1.6880 - learning_rate: 1.0000e-04
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6562 - loss: 1.0872 - val_accuracy: 0.4348 - val_loss: 1.5452 - learning_rate: 1.0000e-04
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.5869 - loss: 1.1129 - val_accuracy: 0.3667 - val_loss: 1.5910 - learning_rate: 1.0000e-04
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7188 - loss: 1.1083 

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

panel = None

# 創建GUI窗口
root = tk.Tk()
root.title("垃圾分類小幫手")

# 設置窗口大小
root.geometry("500x600")

# 上傳圖像按鈕回調函數
def upload_image():
    global panel
    file_path = filedialog.askopenfilename()
    img = Image.open(file_path)
    img = img.resize((256, 256))
    img = ImageTk.PhotoImage(img)
    if panel:
            panel.destroy()
    panel = tk.Label(root, image=img)
    panel.image = img
    panel.pack()
    classify_image(file_path)


# 圖像分類函數
def classify_image(file_path):
    img = Image.open(file_path)
    img = img.resize((256, 256))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    label = list(train_data_gen.class_indices.keys())[predicted_class]
    result_label.config(text=f'分類結果: {label}', font=("Helvetica", 14))

# 設置標題
title_label = tk.Label(root, text="垃圾分類小幫手", font=("Helvetica", 20, "bold"))
title_label.pack(pady=10)

# 創建按鈕
upload_button = tk.Button(root, text="上傳圖片", command=upload_image, font=("Helvetica", 14), bg="#4CAF50", fg="white", relief="groove", padx=10, pady=5)
upload_button.pack(pady=20)

# 創建結果顯示區域
result_label = tk.Label(root, text="分類結果 ", font=("Helvetica", 14))
result_label.pack()

# 啟動GUI循環
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
